# Rating Objective of the Prompt


In [1]:
import pandas as pd
import sqlite3
import os
import numpy as np

from helpers.make_latex_table import create_latex_tables
from helpers.normalization import remove_punctuation, remove_punctuation_and_newlines, remove_newlines
from helpers.statistical_tests import run_t_test_on_gender, compare_genders

db_path = "../../giicg.db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file does not exist: {db_path}")

conn = sqlite3.connect(db_path)
prompts = pd.read_sql("SELECT ap.*, m.message_order from annotated_prompts ap JOIN messages m on ap.message_id = m.message_id ", conn)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set up LLM

In [57]:
# from langchain_openai import ChatOpenAI
# from dotenv import load_dotenv
# from pydantic import BaseModel, Field
# from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
#
# load_dotenv()
#
# class OutputFormat(BaseModel):
#     score: int = Field(description="Final score of the prompt")
#
# system_prompt = SystemMessagePromptTemplate.from_template(
#     "You are an assistant that helps categorize prompts."
# )
#
# user_prompt = HumanMessagePromptTemplate.from_template(
#     """
#     You are a highly experienced judge tasked with evaluating a code generation prompt on criteria.
#     The prompt given to you is provided below:
#     ---
#     {prompt}
#     ---
#     Your task is to evaluate the above prompt on the following criterion on a scale of 1-10:
#     Objectives: How well the prompt explicitly communicates the task objectives, including expected outputs, expected style of the output, formats constraints, audiences, relevant context and the programming language to use.
#     The scoring system is provided below:
#     - 1-2 (Poor): The prompt lacks any clear objectives or guidance.
#     - 3-4 (Below Average): Vague or incomplete objectives.
#     - 5-6 (Average): Outlines basic objectives but lacks depth.
#     - 7-8 (Good): Clearly communicates objectives, may miss edge cases.
#     - 9-10 (Excellent): Comprehensive and leaves no ambiguity.
#
#     Your evaluations must focus on explicit instructions rather than implicit instructions.
#     For example, if the prompt does not mention about the formats or constraints of the objectives
#     then you should not assume that the prompt is effective in communicating the objectives.
#     """,
#
# input_variables=["prompt"]
# )
#
# complete_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
#
# def categorize_prompt(prompt):
#     OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
#     openai_model = "gpt-4.1-2025-04-14"
#     llm = ChatOpenAI(temperature=0.0, model=openai_model)
#     structured_llm = llm.with_structured_output(OutputFormat)
#
#     print(f"evaluating next prompt")
#     chain_one = (
#             {"prompt": lambda x: x["prompt"]}
#             | complete_prompt
#             | structured_llm
#             | {"score": lambda x: x.score}
#     )
#
#     response =  chain_one.invoke({"prompt": prompt})
#
#     return response["score"]

In [58]:
# prompts['objectives_score_3'] = prompts['message_text'].apply(categorize_prompt)
# prompts

In [59]:
#prompts = prompts.loc[:, ~prompts.columns.duplicated()]


## Save to Database

In [60]:
#prompts.to_sql("annotated_prompts", conn, if_exists="replace")

In [61]:
# import pandas as pd
#
# def majority_vote(row):
#     values = [row['objectives_score'], row['objectives_score_2'], row['objectives_score_3']]
#     counts = pd.Series(values).value_counts()
#     # Return the value with the highest count (majority). If tie, return the smallest value (or implement other logic).
#     return int(counts.index[0])
#
#
# # Apply to DataFrame:
# prompts['maj_objectives_score'] = prompts.apply(majority_vote, axis=1)
# prompts

,level_0,index,message_id,conversation_id,role,message_text,gender,user_id,language,ambiguity,...,flip_roles,questions_about_output,points_to_errors,age,work_exp_years,objectives_score,objectives_score_2,objectives_score_3,maj_objectives_score,message_order
0,0.0,0.0,1,1,user,"parsing data from python iterator, how it coul...",Man (cisgender),6,en,1,...,0,0,0.0,19-25,3.0,3,3,3,3,0
1,1.0,1.0,730,32,user,Write python function to do operations with in...,Man (cisgender),6,en,0,...,0,0,0.0,19-25,3.0,5,5,5,5,0
2,2.0,2.0,1133,55,user,Write shortest tutorial on creating RAG on ema...,Man (cisgender),6,en,0,...,0,0,0.0,19-25,3.0,3,3,3,3,0
3,3.0,3.0,1135,55,user,what is FAISS,Man (cisgender),6,en,0,...,0,0,0.0,19-25,3.0,2,1,1,1,2
4,4.0,4.0,1137,55,user,Transform given code to process large .mbox file,Man (cisgender),6,en,0,...,0,0,0.0,19-25,3.0,3,3,3,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,748.0,748.0,1131,54,user,import pandas as pd\nimport numpy as np\nfrom ...,Man (cisgender),92,en,0,...,0,0,0.0,36-40,3.0,5,5,5,5,0
748,749.0,749.0,1532,71,user,"from transformers import AutoTokenizer, AutoMo...",Man (cisgender),92,en,0,...,0,0,0.0,36-40,3.0,4,4,4,4,0
749,750.0,750.0,1646,82,user,"def run_query(query, n_results):\n query_em...",Man (cisgender),92,en,1,...,0,0,NaN,36-40,3.0,4,4,4,4,0
750,751.0,751.0,1849,2,user,\n I am working on the problem of reconstru...,Man (cisgender),8,en,0,...,0,0,0.0,19-25,1.0,8,7,7,7,0


In [2]:
from statsmodels.stats.multitest import multipletests


def groupwise_comparison_with_correction(df, columns, one_sided=False, direction="greater"):
    stat_vals = []
    dfs= []
    p_vals = []
    test_types = []
    directions = []
    one_sideds = []
    female_means = []
    male_means = []
    female_stds = []
    male_stds = []
    n_females = []
    n_males = []
    nonzero_females = []
    nonzero_males = []
    effsizes = []
    effsize_types = []
    cis = []
    for col in columns:
        result = compare_genders(df, col, one_sided=one_sided, direction=direction)
        stat_vals.append(result['test_statistic'])
        dfs.append(result['df'])
        p_vals.append(result['p_value'])
        test_types.append(result['test_type'])
        directions.append(result['direction'])
        one_sideds.append(result['one_sided'])
        female_means.append(result['m_female'])
        male_means.append(result['m_male'])
        female_stds.append(result['std_female'])
        male_stds.append(result['std_male'])
        n_females.append(result['n_female'])
        n_males.append(result['n_male'])
        nonzero_females.append(result['hits_female'])
        nonzero_males.append(result['hits_male']),
        effsizes.append(result['effect_size']),
        effsize_types.append(result['effect_size_type'])
        cis.append(result['effect_size_CI'])
    reject, pvals_corrected, _, _ = multipletests(p_vals, alpha=0.05, method='fdr_bh')

    results = pd.DataFrame({
        'word': columns,
        'test type': test_types,
        'direction': directions,
        'one_sided': one_sideds,
        'df': dfs,
        'stat_value': stat_vals,
        'p_value': p_vals,
        'corrected p_value': pvals_corrected,
        'mean_f': female_means,
        'mean_m': male_means,
        'std_f': female_stds,
        'std_m': male_stds,
        'n_f': n_females,
        'n_m': n_males,
        'n_hits_f': nonzero_females,
        'n_hits_m': nonzero_males,
        'effsize': effsizes,
        'effsize_type': effsize_types,
        'ci': cis,
    })
    return results

In [3]:
user_prompts = (
    prompts
    .fillna({'work_exp_years': 'None'})
    .groupby(['user_id', 'gender', 'age', 'work_exp_years'])['maj_objectives_score']
    .mean()
    .reset_index()    # Reset index to create a DataFrame
)
user_prompts

,user_id,gender,age,work_exp_years,maj_objectives_score
0,6,Man (cisgender),19-25,3.0,2.444444
1,8,Man (cisgender),19-25,1.0,4.500000
2,11,Woman (cisgender),26-30,1.0,3.454545
3,15,Man (cisgender),26-30,6.0,2.333333
4,16,Woman (cisgender),19-25,None,3.208333
5,25,Man (cisgender),26-30,6.0,3.250000
6,28,Woman (cisgender),31-35,5.0,3.095238
7,29,Woman (cisgender),26-30,None,5.000000
8,30,Non-binary,26-30,None,6.000000
9,31,Man (cisgender),36-40,22.0,3.400000


In [7]:
prompts_sorted = prompts.sort_values(['conversation_id', 'message_order'])  # or whatever designates order

# Get the first prompt in each conversation
first_prompts = prompts_sorted.groupby('conversation_id').first().reset_index()

# Now group by user and aggregate (e.g., mean, sum)
user_first_prompts = (
    first_prompts
    .fillna({'work_exp_years': 'None'})
    .groupby(['user_id', 'gender', 'age', 'work_exp_years'])['maj_objectives_score']
    .mean()
    .reset_index()
)

user_prompts_comms = (
    prompts
    .fillna({'work_exp_years': 'None'})
    .groupby(['user_id', 'gender', 'age', 'work_exp_years'])['maj_communication_score']
    .mean()
    .reset_index()    # Reset index to create a DataFrame
)

combined = pd.concat([user_first_prompts, user_prompts_comms['maj_communication_score']], axis=1)
combined


,user_id,gender,age,work_exp_years,maj_objectives_score,maj_communication_score
0,6,Man (cisgender),19-25,3.0,3.666667,4.666667
1,8,Man (cisgender),19-25,1.0,7.000000,5.500000
2,11,Woman (cisgender),26-30,1.0,4.400000,5.454545
3,15,Man (cisgender),26-30,6.0,2.000000,4.000000
4,16,Woman (cisgender),19-25,None,4.500000,4.875000
5,25,Man (cisgender),26-30,6.0,4.000000,5.250000
6,28,Woman (cisgender),31-35,5.0,4.750000,5.000000
7,29,Woman (cisgender),26-30,None,7.000000,4.500000
8,30,Non-binary,26-30,None,6.000000,5.000000
9,31,Man (cisgender),36-40,22.0,4.666667,6.800000


In [14]:
from helpers.make_latex_table import create_latex_tables
result = groupwise_comparison_with_correction(combined, ['maj_objectives_score','maj_communication_score'])
create_latex_tables(result, "comms_obj_stats.tex", "comms_obj_stats_effectsize.tex")
result

,word,test type,direction,one_sided,df,stat_value,p_value,corrected p_value,mean_f,mean_m,std_f,std_m,n_f,n_m,n_hits_f,n_hits_m,effsize,effsize_type,ci
0,maj_objectives_score,Mann-Whitney U,greater,False,NaN,126.000000,0.195820,0.195820,3.666667,4.333333,1.074361,1.459993,13,15,13,15,0.292308,RBC,"[-0.14, 0.68]"
1,maj_communication_score,T-test,two-sided,False,25.778054,2.386308,0.024656,0.049311,4.265074,4.964862,0.753454,0.796817,13,15,13,15,0.900508,Cohen's d,"[0.22, 1.71]"
